In [ ]:
def all_distance(flist):
    '''Returns closest bus stop from each HDB block in Singapore'''
    # For each flight
    all_dist = {}
    min_dist = {}
    alldist_df = pd.DataFrame()
    for flight, fdata in flist.items():
        print(f"Working on flight {flight}")

        # If it is the first 2 trials (South of runway)
        if eval(flight[-1]) < 3:
            long1, lat1 = obs1

        # Else 3rd trial (north of runway)
        else:
            # Different obs1erver point from Nov onwards.
            long1, lat1 = obs2
        print(f"obs1erver is at {long1}, {lat1}")

        rowcount = 0
        dropcount = 0

        # 2nd layer
        all_dist[flight] = {}
        min_dist[flight] = {}

        # Iterate by rows (timestamps)
        # Row is stored as tuples.
        num = 0
        margin_h = 0.006
        margin_v = 0.006  # of a degree. 1 degree = 111000m
        # 0.004 = 444m region

        # Filter out suitable rows.

        orgrowcount = fdata.shape[0]
        print("After filter:")
        fdata = fdata[(abs(fdata['Long']-long1) < margin_h)
                      & (abs(fdata['Lat']-lat1) < margin_v) & (fdata['Altitude']>0)]
        newrowcount = fdata.shape[0]
        dropcount = orgrowcount-newrowcount
        print(fdata)
        # print(f"Suitable data are:\n {fdata[['Long','Lat','Callsign']]}")

        # Interpolation added
        fdata = interpolate(fdata)
        print("Exiting interpolation...")

        # Go row-by-row for each flight's complete data (w/ interpolate)
        for row in fdata.iterrows():
            rowdata = pd.DataFrame(row[1]).T

            # rowdata = pd.DataFrame(fdata.iloc[num])
            # rowdata = rowdata.T #Transpose to vertical
            # print(f"The current row's data is:\n{rowdata}")
            alt = row[1].loc['Altitude']*0.3048
            # print(f"Altitude: {alt}m")
            long, lat = float(row[1].Long), float(row[1].Lat)

            timestamp = row[1].UTC
            date = row[1].UTC[:10]
            # print(f"The date is {date}")
            # print(f"Longitude is {long}, latitude is {lat}")

            dist = math.sqrt(ellipdist(long1, lat1, long, lat)**2+alt**2)
            dist_ft = round(dist/0.3048, 1)
            rowdata.insert(loc=0, column='Distance (ft)', value=dist_ft)
            # rowdata.insert(loc=1, column='Time',value=timestamp)
            # rowdata.insert(loc=1, column='Date',value=date)

            all_dist[flight][timestamp] = dist

            # print(f"Rowdata updated is:\n {rowdata}")
            # print(f"\nRowdata is:\n{rowdata}")
            # print(f"\nalldist_df after updating {flight} is:\n{alldist_df}")

            alldist_df = pd.concat([alldist_df, rowdata])
            rowcount += 1
            num += 1

        try:
            # Alt:
            global dist923
            global dist1014
            global dist1125
            global dist1230
            global dist0127
            dist923 = alldist_df[alldist_df['Date'] == '2022-09-23']
            dist1014 = alldist_df[alldist_df['Date'] == '2022-10-14']
            dist1125 = alldist_df[alldist_df['Date'] == '2022-11-25']
            dist1230 = alldist_df[alldist_df['Date'] == '2022-12-30']
            # print(f"dist923 is: \n{dist923}")
            dist0127 = alldist_df[alldist_df['Date'] == '2023-01-27']

            # Compiles the indices of minimum distances
            # for each flight (for ecah date) for easy selection.
            idx1 = dist923.groupby(['Callsign'])['Distance (ft)'].transform(
                min) == dist923['Distance (ft)']
            idx2 = dist1014.groupby(['Callsign'])['Distance (ft)'].transform(
                min) == dist1014['Distance (ft)']
            idx3 = dist1125.groupby(['Callsign'])['Distance (ft)'].transform(
                min) == dist1125['Distance (ft)']
            idx4 = dist1230.groupby(['Callsign'])['Distance (ft)'].transform(
                min) == dist1230['Distance (ft)']
            idx5 = dist0127.groupby(['Callsign'])['Distance (ft)'].transform(
                min) == dist0127['Distance (ft)']
            # if not dist1014.empty:
            #     prompt = input("Continue for 14 Oct?")
            # if not dist1125.empty:
            #     prompt = input("Continue for 25 Nov?")
            min_distdf = pd.concat(
                [dist923[idx1], dist1014[idx2], dist1125[idx3], dist1230[idx4],dist0127[idx5]])
            print("The list of minimum distances are:")
            print(min_distdf[['Distance (ft)', 'Callsign', 'Long', 'Lat']])
            # minimum = round(min(all_dist[flight].values()),3)
            # min_dist[flight] = minimum
        except ValueError as err:
            sys.exit(f"{err}")
        print(f"{rowcount} rows processed and {dropcount} rows dropped. ")
        # print(f"The closest dist of flight {flight} is:\n {minimum:.2f}m")
    # print(f"The list of min distances are:{min_dist}")
    # min_distdf = pd.DataFrame(min_dist,index=[0])
    return all_dist, alldist_df, min_distdf


'''Compilation and Saving of Distances here'''
all_dist, details, min_distdf = all_distance(flist)
# min_distdf.reset_index(drop=True, inplace=True)

# Inputs and Output Files here
outputfile = 'details 1 Mar.xlsx'
data23sep = '23 Sep details.xlsx'
data14oct = '14 Oct details (3 Nov).xlsx'
data25nov = '25 Nov details.xlsx'
data30dec = '30 Dec details.xlsx'
data27jan = '27 Jan details.xlsx'

# Chronological
filelist = [data23sep, data14oct, data25nov, data30dec, data27jan]
min_distdf.to_excel(outputfile, index=False)


In [2]:
import pandas as pd
import ast
#f = open("./busstopv2.txt", "r")
#pd.read_csv(pd.compat.StringIO("\n".join(lines)), sep=";")

In [3]:
with open("busstopv2.txt", "r") as grilled_cheese:
    lines = grilled_cheese.readlines()
    i=0
    collector = []
    for item in lines:
        if i < 1000:
            collector.append(ast.literal_eval(item))
            #df = dict(item)
            #print(df)
            #print(type(df))
            i+=1
    print(len(collector))   
    #dictlines = dict(lines)
    #print(dictlines["value"])

11


In [29]:
i=2
df = pd.DataFrame(collector[i]['value'],index=[i for i in range(len(collector[1]['value']))])
print(df)

    BusStopCode        RoadName        Description  Latitude   Longitude
0         22161  Pioneer Rd Nth   Aft Lok Yang Way  1.325517  103.696431
1         22169  Pioneer Rd Nth        Aft Intl Rd  1.326245  103.696616
2         22171  Pioneer Rd Nth  Opp Secura S'pore  1.329542  103.696377
3         22179  Pioneer Rd Nth      Secura S'Pore  1.330005  103.696659
4         22181  Pioneer Rd Nth        S'pore Post  1.333016  103.696377
..          ...             ...                ...       ...         ...
495       28071    Boon Lay Way            Blk 105  1.337650  103.739042
496       28079    Boon Lay Way            Blk 210  1.338399  103.738601
497       28081    Boon Lay Way            Blk 112  1.339508  103.736648
498       28089    Boon Lay Way       Bef Yuhua CC  1.340164  103.736346
499       28091    Boon Lay Way       Lakeside Stn  1.344258  103.720125

[500 rows x 5 columns]


In [33]:
import openpyxl
dfs = []
for i in range(len(collector)):
    dfs.append(pd.DataFrame(collector[i]['value'],index=[i for i in range(len(collector[i]['value']))]))
print(f"Done with appending {i} DataFrames")
for i in range(len(collector)):
    #pass
    print(f"Dataframe {i+1}:\n")
    print(dfs[i])
    
overall = pd.concat(dfs,axis=0)
print("Overall DataFrame:")
print(overall.shape)


Done with appending 10 DataFrames
Dataframe 1:

    BusStopCode        RoadName                Description  Latitude  \
0         01012     Victoria St        Hotel Grand Pacific  1.296848   
1         01013     Victoria St            St. Joseph's Ch  1.297710   
2         01019     Victoria St            Bras Basah Cplx  1.296990   
3         01029   Nth Bridge Rd               Opp Natl Lib  1.296673   
4         01039   Nth Bridge Rd                 Bugis Cube  1.298208   
..          ...             ...                        ...       ...   
495       14019     Kg Bahru Rd         Opp Mt Faber Lodge  1.268489   
496       14021  Lower Delta Rd       The Pearl @ Mt Faber  1.271330   
497       14029  Lower Delta Rd                    Blk 105  1.272473   
498       14031  Lower Delta Rd      St. Theresa's Convent  1.276243   
499       14039  Lower Delta Rd  Opp St. Theresa's Convent  1.276636   

      Longitude  
0    103.852536  
1    103.853225  
2    103.853022  
3    103.854414

In [34]:
from openpyxl.utils.dataframe import dataframe_to_rows
wb = Workbook()
ws = wb.active

# For each row in DataFrame
for r in dataframe_to_rows(overall, index=True, header=True):
    ws.append(r)

for cell in ws['A'] + ws[1]:
    cell.style = 'Pandas'

wb.save("busstop_overall.xlsx")

NameError: name 'Workbook' is not defined